# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [3]:
# your code here
#Trabajo con la bd de las chicas
url='https://www.zalando.fr/api/catalog/articles?categories=promo-femme&limit=84&offset=0&sort=popularity'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [6]:
# your code here
#Trabajo con la bd de las chicas
import json
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import re

url='https://www.zalando.fr/api/catalog/articles?categories=promo-femme&limit=84&offset=0&sort=popularity'
response = requests.get(url, headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"})
result=response.json()

pag=result['pagination']
page_count=pag['page_count']
per_page=pag['per_page']

dfTotalArticles=pd.DataFrame()
for x in range(0,page_count):
    url="https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset="+ str(int(per_page)*x)+"&sort=popularity"
    response = requests.get(url, headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"})
    result=response.json()
    if x==0:
        dfTotalArticles=json_normalize(json_normalize(result).articles[0])
    else:
        dfTotalArticles=pd.concat([dfTotalArticles,json_normalize(json_normalize(result).articles[0])], ignore_index=True, sort=True)

print(dfTotalArticles.columns)
dfTotalArticles.columns=[str.strip(x.lower()) for x in dfTotalArticles.columns]
dfTotalArticles.columns=[re.sub(r"[. \ :+]","_",x) for x in dfTotalArticles.columns]
dfTotalArticles.columns=[re.sub("__","_",x) for x in dfTotalArticles.columns]
print(dfTotalArticles.columns)
dfTotalArticles.head()

Index(['amount', 'brand_name', 'delivery_promises', 'family_articles', 'flags',
       'is_premium', 'media', 'name', 'price.base_price',
       'price.has_different_original_prices', 'price.has_different_prices',
       'price.has_different_promotional_prices',
       'price.has_discount_on_selected_sizes_only', 'price.original',
       'price.promotional', 'product_group', 'sizes', 'sku',
       'tracking_information.impression_beacon',
       'tracking_information.metrigo_impression_urls',
       'tracking_information.source', 'url_key'],
      dtype='object')
Index(['amount', 'brand_name', 'delivery_promises', 'family_articles', 'flags',
       'is_premium', 'media', 'name', 'price_base_price',
       'price_has_different_original_prices', 'price_has_different_prices',
       'price_has_different_promotional_prices',
       'price_has_discount_on_selected_sizes_only', 'price_original',
       'price_promotional', 'product_group', 'sizes', 'sku',
       'tracking_information_impress

,amount,brand_name,delivery_promises,family_articles,flags,is_premium,media,name,price_base_price,price_has_different_original_prices,...,price_has_discount_on_selected_sizes_only,price_original,price_promotional,product_group,sizes,sku,tracking_information_impression_beacon,tracking_information_metrigo_impression_urls,tracking_information_source,url_key
0,NaN,Nike Performance,[],"[{'sku': 'N1243A0XO-Q11', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': '-10%', 'tra...",False,[{'path': 'N1/24/3A/0X/OQ/11/N1243A0XO-Q11@11....,TEAM HUSTLE QUICK 2 - Chaussures de basket - b...,NaN,False,...,False,"34,95 €","31,45 €",shoe,"[27.5, 28, 28.5, 29.5, 30, 31, 32, 33, 33.5, 3...",N1243A0XO-Q11,https://ccp-et.metrigo.zalan.do/event/sbv?z=af...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=a...,ccp,nike-performance-team-hustle-quick-2-chaussure...
1,NaN,Tommy Hilfiger,[],"[{'sku': 'TO113I00A-Q11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': '-20%', 'tra...",False,[{'path': 'TO/11/3I/00/AQ/11/TO113I00A-Q11@10....,BOOT - Bottines à lacets - black,NaN,False,...,False,"99,95 €","79,95 €",shoe,"[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 3...",TO113I00A-Q11,https://ccp-et.metrigo.zalan.do/event/sbv?z=af...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=a...,ccp,tommy-hilfiger-boot-bottines-a-lacets-black-to...
2,NaN,Tommy Hilfiger,[],"[{'sku': 'TO113I00G-K11', 'url_key': 'tommy-hi...","[{'key': 'discountRate', 'value': '-30%', 'tra...",False,[{'path': 'TO/11/3I/00/GK/11/TO113I00G-K11@8.1...,Bottines - blue,NaN,False,...,False,"94,95 €","66,45 €",shoe,"[28, 29, 30, 33, 34, 35, 36, 37, 38, 40]",TO113I00G-K11,https://ccp-et.metrigo.zalan.do/event/sbv?z=af...,[https://ccp-et.metrigo.zalan.do/event/sbv?z=a...,ccp,tommy-hilfiger-bottines-blue-to113i00g-k11
3,NaN,Nike Performance,[],"[{'sku': 'N1243D0ZI-Q11', 'url_key': 'nike-per...","[{'key': 'discountRate', 'value': '-10%', 'tra...",False,[{'path': 'N1/24/3D/0Z/IQ/11/N1243D0ZI-Q11@4.j...,DRY ACADEMY DRILL TOP - T-shirt de sport - bla...,NaN,False,...,False,"34,95 €","31,45 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",N1243D0ZI-Q11,NaN,NaN,NaN,nike-performance-dry-dril-t-shirt-a-manches-lo...
4,NaN,Vans,[],"[{'sku': 'VA224G00U-Q11', 'url_key': 'vans-t-s...","[{'key': 'discountRate', 'value': '-10%', 'tra...",False,[{'path': 'VA/22/4G/00/UQ/11/VA224G00U-Q11@6.1...,CLASSIC BOYS - T-shirt imprimé - black/white,NaN,False,...,False,"19,95 €","17,95 €",clothing,"[8-10a, 10-12a, 14-16a, 16-18a]",VA224G00U-Q11,NaN,NaN,NaN,vans-t-shirt-imprime-blackwhite-va224g00u-q11


In [7]:
dfTotalArticles.sku.count()

21968

## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [8]:
# your code here
def clean_number(number):
    clean_number=re.sub("\D+", "", str(number))
    return clean_number

def clean_text(text):
    clean_text=re.sub("\W+\s*", " ", str(text))
    return clean_text

def clean_path(txt):
    clean_packet_shot=txt['packet_shot']
    clean_path=txt['path']
    return (clean_path,clean_packet_shot)

def clean_float(float_v):
    float_v = re.sub(",", ".", str(float_v))
    clean_float=re.match("\d+\.?\d*", float_v).group()
    return clean_float

dfTotalArticles['amount_c']= dfTotalArticles.amount.apply(lambda x:clean_number(x))
dfTotalArticles['brand_name_c']= dfTotalArticles.brand_name.apply(lambda x:clean_text(x))
dfTotalArticles['delivery_promises_c']= dfTotalArticles.delivery_promises.apply(lambda x:clean_text(x))
dfTotalArticles[['path_c','packet_shot_c']]= dfTotalArticles.media.apply(lambda x:clean_path(x[0])).apply(pd.Series)
dfTotalArticles['price_original_c']= dfTotalArticles.price_original.apply(lambda x:clean_float(x))
dfTotalArticles['price_promotional_c']= dfTotalArticles.price_promotional.apply(lambda x:clean_float(x))
dfOutTotal=dfTotalArticles[['amount_c', 'brand_name_c', 'delivery_promises_c', 'family_articles', 'flags',
       'is_premium', 'path_c','packet_shot_c', 'name', 'price_has_different_original_prices',
       'price_has_different_prices', 'price_has_different_promotional_prices',
       'price_has_discount_on_selected_sizes_only', 'price_original_c',
       'price_promotional_c', 'product_group', 'sizes', 'sku', 'url_key']]
dfOutTotal.columns=['amount', 'brand_name', 'delivery_promises', 'family_articles', 'flags',
       'is_premium', 'path','packet_shot', 'name', 'price_has_different_original_prices',
       'price_has_different_prices', 'price_has_different_promotional_prices',
       'price_has_discount_on_selected_sizes_only', 'price_original',
       'price_promotional', 'product_group', 'sizes', 'sku', 'url_key']

### The trending brand

In [9]:
dfOutTotal.brand_name.value_counts().head(1)

Name it    1024
Name: brand_name, dtype: int64

### The product(s) with the highest discount.

In [14]:
dfOutTotal['discount']=dfOutTotal.price_original.astype(float)-dfOutTotal.price_promotional.astype(float)
dfOutTotal.groupby('sku')['discount'].agg('sum').sort_values(ascending=False).head(1)

C:\Users\Kalilu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


sku
C1824L003-M11    345.0
Name: discount, dtype: float64